In [ ]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 30.3 MB/s 
     |████████████████████████████████| 7.8 MB 58.5 MB/s 
     |████████████████████████████████| 16.6 MB 107 kB/s 
     |████████████████████████████████| 1.6 MB 75.3 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 132 kB 79.0 MB/s 


In [2]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
else:
    !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'RI',
    root_path = root_path,
)

year = 2018
self.get_shapes()#overwrite=True)
self.get_pl()#overwrite=True)
self.get_assignments()#overwrite=True)
self.get_geo()#overwrite=True)
self.get_crosswalks()#overwrite=True)

self.get_acs5(year=year, overwrite=True)
# vp.test.f(5)
# ut.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
shapes.RI2020 ... Elapsed time: 0.3490 seconds
pl.RI2020 ... Elapsed time: 0.1361 seconds
assignments.RI2020 ... Elapsed time: 0.1386 seconds
Elapsed time: 0.1562 seconds
crosswalks.RI ... Elapsed time: 0.0926 seconds
acs5.RI2018_raw fetching ... 

/content/drive/MyDrive/voting_predictor/voting_predictor/main.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['year'] = year
/content/drive/MyDrive/voting_predictor/voting_predictor/main.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[geoid] = ''
/content/drive/MyDrive/helpers/helpers/utilities.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

Can not compute other_vap_spanishathome
Can not compute other_vap_spanishathomeenglishwell


'acs5.RI2018'

In [ ]:
s = 'ababab'
s[s.find('b')+1:]

'abab'

In [ ]:
self.B.head(2)

,year,tract2010,name,b17001h_014e,b01001h_011e,b17001i_011e,b17001h_010e,b01001_039e,b01001h_026e,b17001_012e,...,b01001_025e,b17001h_029e,b25003h_002e,b01001_022e,b01001i_022e,b01001_001e,b01001h_022e,b01001h_008e,b17001_029e,b01001_032e
index,,,,,,,,,,,,,,,,,,,,,
0,2018,44001030100,"Census Tract 301, Bristol County, Rhode Island",9,104,0,15,241,171,22,...,45,8,1483,43,0,4379,14,94,8,15
1,2018,44001030200,"Census Tract 302, Bristol County, Rhode Island",35,165,0,0,98,159,0,...,31,8,907,77,15,3086,23,52,8,0


In [ ]:
A = self.bq.tbl_to_df('redistricting-361203.crosswalks.RI_raw')
B = ut.prep(A)
print(A.dtypes, B.dtypes)

block2010     string
block2020     string
aland          Int64
prop2010     Float64
prop2020     Float64
dtype: object block2010     string
block2020     string
aland          Int64
prop2010     Float64
prop2020     Float64
dtype: object


In [ ]:
features = [
    ['all', 'tot' ,'pop', ['b01001_001e']],
    ['white', 'tot', 'pop', ['b01001h_001e']],
    ['hisp' , 'tot', 'pop', ['b01001i_001e']],


    ['all'  , 'vap', 'pop', [
        'b01001_007e', #pop male 18-19
        'b01001_008e', #pop male 20
        'b01001_009e', #pop male 21
        'b01001_010e', #pop male 22-24
        'b01001_011e', #pop male 25-29
        'b01001_012e', #pop male 30-34
        'b01001_013e', #pop male 35-39
        'b01001_014e', #pop male 40-44
        'b01001_015e', #pop male 45-49
        'b01001_016e', #pop male 50-54
        'b01001_017e', #pop male 55-59
        'b01001_018e', #pop male 60-61
        'b01001_019e', #pop male 62-64
        'b01001_020e', #pop male 65-66
        'b01001_021e', #pop male 67-69
        'b01001_022e', #pop male 70-74
        'b01001_023e', #pop male 75-79
        'b01001_024e', #pop male 80-84
        'b01001_025e', #pop male 85-inf
        'b01001_031e', #pop female 18-19
        'b01001_032e', #pop female 20
        'b01001_033e', #pop female 21
        'b01001_034e', #pop female 22-24
        'b01001_035e', #pop female 25-29
        'b01001_036e', #pop female 30-34
        'b01001_037e', #pop female 35-39
        'b01001_038e', #pop female 40-44
        'b01001_039e', #pop female 45-49
        'b01001_040e', #pop female 50-54
        'b01001_041e', #pop female 55-59
        'b01001_042e', #pop female 60-61
        'b01001_043e', #pop female 62-64
        'b01001_044e', #pop female 65-66
        'b01001_045e', #pop female 67-69
        'b01001_046e', #pop female 70-74
        'b01001_047e', #pop female 75-79
        'b01001_048e', #pop female 80-84
        'b01001_049e', #pop female 85-inf
    ]],
    ['white', 'vap', 'pop', [
        'b01001h_007e', #pop male 18-19
        'b01001h_008e', #pop male 20-24
        'b01001h_009e', #pop male 25-29
        'b01001h_010e', #pop male 30-34
        'b01001h_011e', #pop male 35-44
        'b01001h_012e', #pop male 45-54
        'b01001h_013e', #pop male 55-64
        'b01001h_014e', #pop male 65-74
        'b01001h_015e', #pop male 75-84
        'b01001h_016e', #pop male 85-inf
        'b01001h_022e', #pop female 18-19
        'b01001h_023e', #pop female 20-24
        'b01001h_024e', #pop female 25-29
        'b01001h_025e', #pop female 30-34
        'b01001h_026e', #pop female 35-44
        'b01001h_027e', #pop female 45-54
        'b01001h_028e', #pop female 55-64
        'b01001h_029e', #pop female 65-74
        'b01001h_030e', #pop female 75-84
        'b01001h_031e', #pop female 85-inf
    ]],
    ['hisp' , 'vap', 'pop', [
        'b01001i_007e', #pop male 18-19
        'b01001i_008e', #pop male 20-24
        'b01001i_009e', #pop male 25-29
        'b01001i_010e', #pop male 30-34
        'b01001i_011e', #pop male 35-44
        'b01001i_012e', #pop male 45-54
        'b01001i_013e', #pop male 55-64
        'b01001i_014e', #pop male 65-74
        'b01001i_015e', #pop male 75-84
        'b01001i_016e', #pop male 85-inf
        'b01001i_022e', #pop female 18-19
        'b01001i_023e', #pop female 20-24
        'b01001i_024e', #pop female 25-29
        'b01001i_025e', #pop female 30-34
        'b01001i_026e', #pop female 35-44
        'b01001i_027e', #pop female 45-54
        'b01001i_028e', #pop female 55-64
        'b01001i_029e', #pop female 65-74
        'b01001i_030e', #pop female 75-84
        'b01001i_031e', #pop female 85-inf
    ]],


    ['all'  , 'vap', 'elderly', [
        'b01001_017e', #pop male 55-59
        'b01001_018e', #pop male 60-61
        'b01001_019e', #pop male 62-64
        'b01001_020e', #pop male 65-66
        'b01001_021e', #pop male 67-69
        'b01001_022e', #pop male 70-74
        'b01001_023e', #pop male 75-79
        'b01001_024e', #pop male 80-84
        'b01001_025e', #pop male 85-inf
        'b01001_041e', #pop female 55-59
        'b01001_042e', #pop female 60-61
        'b01001_043e', #pop female 62-64
        'b01001_044e', #pop female 65-66
        'b01001_045e', #pop female 67-69
        'b01001_046e', #pop female 70-74
        'b01001_047e', #pop female 75-79
        'b01001_048e', #pop female 80-84
        'b01001_049e', #pop female 85-inf
    ]],
    ['white', 'vap', 'elderly', [
            'b01001h_013e', #pop male 55-64
            'b01001h_014e', #pop male 65-74
            'b01001h_015e', #pop male 75-84
            'b01001h_016e', #pop male 85-inf
            'b01001h_028e', #pop female 55-64
            'b01001h_029e', #pop female 65-74
            'b01001h_030e', #pop female 75-84
            'b01001h_031e', #pop female 85-inf
    ]],
    ['hisp' , 'vap', 'elderly', [
        'b01001i_013e', #pop male 55-64
        'b01001i_014e', #pop male 65-74
        'b01001i_015e', #pop male 75-84
        'b01001i_016e', #pop male 85-inf  
        'b01001i_028e', #pop female 55-64
        'b01001i_029e', #pop female 65-74
        'b01001i_030e', #pop female 75-84
        'b01001i_031e', #pop female 85-inf
    ]],


    ['all'  , 'vap', 'poverty', [
        'b17001_010e', #poverty male 18-24'
        'b17001_011e', #poverty male 25-34'
        'b17001_012e', #poverty male 35-44'
        'b17001_013e', #poverty male 45-54'
        'b17001_014e', #poverty male 55-64'
        'b17001_015e', #poverty male 65-74'
        'b17001_016e', #poverty male 74-inf'
        'b17001_024e', #poverty female 18-24'
        'b17001_025e', #poverty female 25-34'
        'b17001_026e', #poverty female 35-44'
        'b17001_027e', #poverty female 45-54'
        'b17001_028e', #poverty female 55-64'
        'b17001_029e', #poverty female 65-74'
        'b17001_030e', #poverty female 74-inf'
    ]],
    ['white', 'vap', 'poverty', [
        'b17001h_010e', #poverty male 18-24'
        'b17001h_011e', #poverty male 25-34'
        'b17001h_012e', #poverty male 35-44'
        'b17001h_013e', #poverty male 45-54'
        'b17001h_014e', #poverty male 55-64'
        'b17001h_015e', #poverty male 65-74'
        'b17001h_016e', #poverty male 74-inf'
        'b17001h_024e', #poverty female 18-24'
        'b17001h_025e', #poverty female 25-34'
        'b17001h_026e', #poverty female 35-44'
        'b17001h_027e', #poverty female 45-54'
        'b17001h_028e', #poverty female 55-64'
        'b17001h_029e', #poverty female 65-74'
        'b17001h_030e', #poverty female 74-inf'
    ]],
    ['hisp' , 'vap', 'poverty', [
        'b17001i_010e', #poverty male 18-24'
        'b17001i_011e', #poverty male 25-34'
        'b17001i_012e', #poverty male 35-44'
        'b17001i_013e', #poverty male 45-54'
        'b17001i_014e', #poverty male 55-64'
        'b17001i_015e', #poverty male 65-74'
        'b17001i_016e', #poverty male 74-inf'
        'b17001i_024e', #poverty female 18-24'
        'b17001i_025e', #poverty female 25-34'
        'b17001i_026e', #poverty female 35-44'
        'b17001i_027e', #poverty female 45-54'
        'b17001i_028e', #poverty female 55-64'
        'b17001i_029e', #poverty female 65-74'
        'b17001i_030e', #poverty female 74-inf'
    ]],


    ['all'  , 'vap', 'highschool', [
        # 's1501_c01_003e', #High school graduate (includes equivalency) 18-24
        # 's1501_c01_004e', #Some college or associate's degree 18-24
        # 's1501_c01_005e', #Bachelor's degree or higher 18-24
        's1501_c01_009e', #High school graduate (includes equivalency) 25-inf
        's1501_c01_010e', #Some college, no degree 25-inf
        's1501_c01_011e', #Associate's degree 25-inf
        's1501_c01_012e', #Bachelor's degree 25-inf
        's1501_c01_013e', #Graduate or professional degree 25-inf
    ]],
    ['white', 'vap', 'highschool', ['s1501_c01_032e']],
    ['hisp' , 'vap', 'highschool', ['s1501_c01_053e']],


    ['all'  , 'vap', 'homeowner', ['b25003_002e']],
    ['white', 'vap', 'homeowner', ['b25003h_002e']],
    ['hisp' , 'vap', 'homeowner', ['b25003i_002e']],


    ['hisp' , 'vap', 'spanishAtHome', ['b16004_026e']],
    ['hisp' , 'vap', 'spanishAtHomeEnglishWell', ['b16004_027e', 'b16004_028e']],
]
{f'{x[0]}_{x[1]}_{x[2]}':x[3] for x in features}
# {f'{x[0]}':x[1] for x in features}

{'all_tot_pop': ['b01001_001e'],
 'white_tot_pop': ['b01001h_001e'],
 'hisp_tot_pop': ['b01001i_001e'],
 'all_vap_pop': ['b01001_007e',
  'b01001_008e',
  'b01001_009e',
  'b01001_010e',
  'b01001_011e',
  'b01001_012e',
  'b01001_013e',
  'b01001_014e',
  'b01001_015e',
  'b01001_016e',
  'b01001_017e',
  'b01001_018e',
  'b01001_019e',
  'b01001_020e',
  'b01001_021e',
  'b01001_022e',
  'b01001_023e',
  'b01001_024e',
  'b01001_025e',
  'b01001_031e',
  'b01001_032e',
  'b01001_033e',
  'b01001_034e',
  'b01001_035e',
  'b01001_036e',
  'b01001_037e',
  'b01001_038e',
  'b01001_039e',
  'b01001_040e',
  'b01001_041e',
  'b01001_042e',
  'b01001_043e',
  'b01001_044e',
  'b01001_045e',
  'b01001_046e',
  'b01001_047e',
  'b01001_048e',
  'b01001_049e'],
 'white_vap_pop': ['b01001h_007e',
  'b01001h_008e',
  'b01001h_009e',
  'b01001h_010e',
  'b01001h_011e',
  'b01001h_012e',
  'b01001h_013e',
  'b01001h_014e',
  'b01001h_015e',
  'b01001h_016e',
  'b01001h_022e',
  'b01001h_023e',
 